# B-Tree

Mientras que los RMI son relativamente nuevos y usan modelos predictivos, los B-Trees son una solución probada y robusta para organizar grandes cantidades de datos de forma que las búsquedas, inserciones y eliminaciones sean eficientes, especialmente cuando los datos residen en almacenamiento externo (como discos duros), donde acceder a la información es mucho más lento que en la memoria RAM.

Construiremos un B-Tree clásico en memoria paso a paso.

Es como imaginar que se tiene una biblioteca gigantesca y quieres encontrar un libro rápidamente. Un B-Tree es como un sistema de organización de esa biblioteca, pero muy inteligente. En lugar de una sola lista, organiza los libros en "estantes" y "sub-estantes", permitiéndote llegar al libro deseado con muy pocos pasos.

Las características clave de un B-Tree son:

- Es un árbol equilibrado: Esto significa que, sin importar cuántos datos tenga, la distancia desde la "raíz" (el estante principal) hasta cualquier dato siempre será aproximadamente la misma. Esto garantiza que las búsquedas sean rápidas y predecibles.
- Cada "nodo" (estante) puede contener múltiples claves (libros) y múltiples hijos (sub-estantes): A diferencia de otros árboles donde cada nodo tiene solo dos hijos, en un B-Tree, un nodo puede tener muchos hijos. Esto es crucial para su eficiencia en sistemas de almacenamiento externo, ya que reduce el número de "viajes" al disco.
- Los nodos están ordenados: Las claves dentro de cada nodo y los valores en los hijos están estrictamente ordenados. Esto permite búsquedas rápidas dentro de cada nodo.
- Minimiza las operaciones de disco: Los B-Trees están diseñados para reducir al mínimo la cantidad de veces que se debe leer o escribir información en el disco, lo que los hace ideales para bases de datos. Aunque nuestra implementación será en memoria, el concepto sigue siendo el mismo.

#### Orden del B-Tree (t)

Un concepto fundamental en los B-Trees es su orden (t). Este número define el número mínimo y máximo de claves que un nodo puede contener, y el número de hijos que puede tener.

- Cada nodo (excepto la raíz) debe tener al menos t−1 claves.
- Cada nodo puede tener como máximo 2t−1 claves.
- Cada nodo con k claves tiene k+1 hijos.

Por ejemplo, si t=3:

- Un nodo debe tener al menos 3−1=2 claves.
- Un nodo puede tener como máximo 2×3−1=5 claves.
- Un nodo con 5 claves tendrá 6 hijos.

Componentes de un B-Tree

Para construir nuestro B-Tree, necesitamos dos componentes principales:

- Clase BTreeNode (El "Estante"): Representará cada nodo individual del árbol. Contendrá:
    - Una lista de claves (los números que almacenamos).
    - Una lista de hijos (referencias a otros BTreeNodes, los "sub-estantes").
    - Un indicador para saber si es una hoja (el "último estante" que contiene los datos finales y no tiene sub-estantes).
    - La referencia al padre (opcional, pero útil para ciertas operaciones).

- Clase BTree (La "Biblioteca Completa"): Representará el árbol en su totalidad. Contendrá:
    - La raíz del árbol (el "estante principal").
    - El orden (t) del árbol

Preparación: Importaciones y Orden del Árbol

Necesitaremos numpy para generar datos y algunas funciones auxiliares si decidimos usarlas de utils.py (aunque para esta demo, mantendremos todo autocontenido para mayor claridad).

In [1]:
import numpy as np

# Definimos el orden de nuestro B-Tree
# Este es un parámetro crucial que afecta el rendimiento y el tamaño del nodo.
# t=3 significa que cada nodo tendrá entre 2 y 5 claves.
T = 3

1. La Clase BTreeNode (El Nodo del Árbol)

Comenzaremos definiendo la estructura de un solo nodo de nuestro B-Tree

In [2]:
# --- Clase BTreeNode ---

class BTreeNode:
    def __init__(self, t, is_leaf):
        """
        Constructor para un nodo de B-Tree.

        Args:
            t (int): El orden del B-Tree (número mínimo de hijos que puede tener un nodo interno).
            is_leaf (bool): True si este nodo es una hoja, False en caso contrario.
        """
        self.t = t                      # Orden del árbol
        self.is_leaf = is_leaf          # ¿Es una hoja? (no tiene hijos)
        self.keys = []                  # Lista de claves (datos) en el nodo
        self.children = []              # Lista de hijos (otros BTreeNode)
        self.n_keys = 0                 # Número actual de claves en el nodo

    def __str__(self):
        """Representación en cadena del nodo para depuración."""
        return f"Node(keys={self.keys}, is_leaf={self.is_leaf})"

2. La Clase BTree (La Estructura del Árbol)

Ahora, definiremos la clase principal que contendrá nuestro B-Tree. Aquí manejaremos la raíz del árbol y las operaciones de alto nivel como la búsqueda y la inserción.

In [5]:
# --- Clase BTree ---

class BTree:
    def __init__(self, t):
        """
        Constructor para el B-Tree.

        Args:
            t (int): El orden del B-Tree.
        """
        self.t = t                      # Orden del árbol
        self.root = BTreeNode(t, True)  # La raíz del árbol, inicialmente una hoja vacía.

    def search(self, key):
        """
        Busca una clave en el B-Tree.

        Args:
            key: La clave a buscar.

        Returns:
            tuple: (nodo, índice) si la clave se encuentra, donde nodo es el BTreeNode
                   que contiene la clave e índice es la posición de la clave dentro de ese nodo.
                   Retorna None si la clave no se encuentra.
        """
        return self._search_recursive(self.root, key)

    def _search_recursive(self, node, key):
        """
        Función auxiliar recursiva para la búsqueda.
        """
        i = 0
        # Mueve 'i' a la primera clave mayor o igual que la clave buscada
        while i < node.n_keys and key > node.keys[i]:
            i += 1

        # Si encontramos la clave en este nodo
        if i < node.n_keys and key == node.keys[i]:
            return node, i # Clave encontrada en este nodo, en esta posición

        # Si la clave no está en este nodo y es una hoja, no la encontraremos
        if node.is_leaf:
            return None, None # Clave no encontrada

        # Si no es una hoja, bajamos al hijo apropiado
        return self._search_recursive(node.children[i], key)

    def insert(self, key):
        """
        Inserta una clave en el B-Tree.
        """
        root_node = self.root
        # Si la raíz está llena, el árbol crece en altura.
        # Esto es una operación de "split" en la raíz.
        if root_node.n_keys == (2 * self.t - 1):
            s = BTreeNode(self.t, False) # Crea un nuevo nodo raíz no-hoja
            s.children.insert(0, root_node) # El viejo raíz se convierte en su primer hijo
            self._split_child(s, 0) # Divide el viejo raíz
            self.root = s # La nueva raíz es 's'
            self._insert_non_full(s, key) # Inserta la clave en la nueva estructura
        else:
            # Si la raíz no está llena, simplemente inserta.
            self._insert_non_full(root_node, key)

    def _insert_non_full(self, node, key):
        """
        Inserta una clave en un nodo que no está lleno.
        Esta es la parte recursiva que baja por el árbol.
        """
        i = node.n_keys - 1
        if node.is_leaf:
            # Si es una hoja, encontramos la posición correcta e insertamos la clave.
            node.keys.append(None) # Agrega un espacio al final para la nueva clave
            while i >= 0 and key < node.keys[i]:
                node.keys[i + 1] = node.keys[i]
                i -= 1
            node.keys[i + 1] = key
            node.n_keys += 1
        else:
            # Si no es una hoja, encontramos el hijo correcto para bajar.
            while i >= 0 and key < node.keys[i]:
                i -= 1
            i += 1 # Ahora 'i' es el índice del hijo donde debemos ir

            # Si el hijo está lleno, lo dividimos antes de bajar.
            if node.children[i].n_keys == (2 * self.t - 1):
                self._split_child(node, i) # Divide el hijo
                # Después de dividir, la clave mediana sube al padre (node).
                # Necesitamos decidir si la nueva clave va a la izquierda o derecha de la clave ascendida.
                if key > node.keys[i]:
                    i += 1 # Si es mayor, va al hijo de la derecha del nuevo padre
            
            # Recursivamente inserta en el hijo apropiado
            self._insert_non_full(node.children[i], key)

    def _split_child(self, parent_node, i):
        """
        Divide el i-ésimo hijo del nodo padre que está lleno.
        La clave mediana del hijo se mueve al nodo padre.
        """
        t = self.t
        full_child = parent_node.children[i]
        new_child = BTreeNode(t, full_child.is_leaf) # El nuevo hijo tendrá las claves "grandes"

        # La clave mediana del hijo "full_child" asciende al padre
        median_key = full_child.keys[t - 1]

        # Las claves a la derecha de la mediana van al nuevo_child
        new_child.keys = full_child.keys[t:]
        full_child.keys = full_child.keys[:t-1] # El hijo original se queda con las claves "pequeñas"

        # Actualiza el número de claves
        full_child.n_keys = t - 1
        new_child.n_keys = t - 1 # El nuevo hijo también tendrá t-1 claves al principio

        # Si el hijo original no es una hoja, sus hijos también se dividen
        if not full_child.is_leaf:
            new_child.children = full_child.children[t:]
            full_child.children = full_child.children[:t]

        # Inserta la clave mediana en el padre
        parent_node.keys.insert(i, median_key)
        # Inserta el nuevo hijo en la lista de hijos del padre
        parent_node.children.insert(i + 1, new_child)
        parent_node.n_keys += 1 # El padre gana una clave

3. Demostración de B-Tree: Construcción y Búsqueda

Ahora, vamos a ver nuestro B-Tree en acción. Insertaremos una serie de números y luego buscaremos algunos de ellos para verificar su funcionalidad.

In [6]:
# --- Demostración de B-Tree ---

print(f"Creando un B-Tree con orden t = {T}")
b_tree = BTree(T)

# Datos para insertar (los desordenamos para simular inserciones reales)
keys_to_insert = [10, 20, 5, 30, 15, 25, 40, 0, 35, 7, 22, 18, 50, 45, 12, 28, 33]
# Para una mejor demostración, ordenamos los keys_to_insert para que el árbol
# no tenga que hacer muchos splits al inicio, pero en un uso real, las inserciones
# pueden ser en cualquier orden.
keys_to_insert.sort() 
# Para una demo de splits mejor, podemos insertar en orden ascendente y luego algunos aleatorios
# keys_to_insert = list(range(1, 51, 3)) + [2, 17, 48, 1] # Ejemplo con desorden

print(f"Insertando las siguientes claves: {keys_to_insert}")
for key in keys_to_insert:
    b_tree.insert(key)
    # Opcional: imprimir el árbol para ver cómo crece (requiere una función de impresión más elaborada)
    # print(f"Después de insertar {key}:")
    # print_b_tree(b_tree.root) # Una función de impresión que no hemos implementado aquí, pero sería útil.

print("\n--- Realizando búsquedas en el B-Tree ---")

search_values = [15, 25, 1, 50, 2, 33, 100] # Claves a buscar

for val in search_values:
    node, idx = b_tree.search(val)
    if node is not None:
        print(f"Clave {val} encontrada en el nodo: {node.keys[idx]} (en índice {idx} del nodo)")
    else:
        print(f"Clave {val} NO encontrada en el B-Tree.")

Creando un B-Tree con orden t = 3
Insertando las siguientes claves: [0, 5, 7, 10, 12, 15, 18, 20, 22, 25, 28, 30, 33, 35, 40, 45, 50]

--- Realizando búsquedas en el B-Tree ---
Clave 15 encontrada en el nodo: 15 (en índice 1 del nodo)
Clave 25 encontrada en el nodo: 25 (en índice 0 del nodo)
Clave 1 NO encontrada en el B-Tree.
Clave 50 encontrada en el nodo: 50 (en índice 4 del nodo)
Clave 2 NO encontrada en el B-Tree.
Clave 33 encontrada en el nodo: 33 (en índice 0 del nodo)
Clave 100 NO encontrada en el B-Tree.


Un Breve Análisis de la Inserción y el _split_child

La operación más compleja en un B-Tree es la inserción, especialmente cuando un nodo se llena y debe dividirse.

La función _split_child(parent_node, i) es el corazón de cómo un B-Tree mantiene su equilibrio y profundidad baja:

- Cuando un nodo hijo está a punto de llenarse (contiene 2*t - 1 claves y se le va a insertar una nueva), el padre lo "divide".
- La clave mediana del hijo lleno (la clave en la posición t-1) es promovida y se inserta en el nodo padre.
- Las claves a la izquierda de la mediana se quedan en el hijo original.
- Las claves a la derecha de la mediana se mueven a un nuevo nodo hijo.
- El nuevo nodo hijo se añade como un nuevo hijo del padre.

Este proceso asegura que el árbol siempre esté balanceado y que la altura se mantenga al mínimo, garantizando búsquedas rápidas. Si la raíz se llena, ella misma se divide, creando una nueva raíz y aumentando la altura del árbol en uno.
Consideraciones sobre la Implementación de un B-Tree

- Complejidad: Implementar un B-Tree completo con todas las operaciones (inserción, eliminación, búsqueda) puede ser bastante complejo debido a la necesidad de mantener el balanceo del árbol. Aquí nos hemos centrado en la búsqueda y la inserción, que son las más relevantes para la comparación con RMI.
- Eficiencia en Disco: Aunque nuestra implementación es en memoria, el diseño del B-Tree está optimizado para accesos a disco. Cada nodo es típicamente del tamaño de un bloque de disco (o un múltiplo de él) para minimizar las operaciones de E/S (Input/Output).
- Comparación con B+Tree: Una variante común y a menudo más eficiente para bases de datos es el B+Tree. En un B+Tree, solo los nodos hoja contienen los datos reales (o punteros a ellos), y todos los nodos hoja están enlazados entre sí, lo que facilita las búsquedas de rango. Los nodos internos solo contienen claves de enrutamiento para guiar la búsqueda. Para este proyecto, un B-Tree clásico es suficiente para la comparativa básica